In [1]:
from copy import deepcopy

import numpy as np
import open3d as o3d
import trimesh
from open3d.examples.visualization.to_mitsuba import dataset

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


simplify_vertex_clustering is not good for subdivide surface fitting, simplify_quadric_decimation is much better
notes that boundary_weight will influence the result of decimated mesh, inappropriate setting leads to holes in the mesh.

In [9]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[np.asarray(index)]
        
    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [10]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(1,16):
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Mitch\output/deformed_{i:03}.obj')
    mesh.compute_vertex_normals()
    deformed_meshes.append(mesh)
    all_meshes += mesh
o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())

TriangleMesh with 224896 points and 450000 triangles. True
PointCloud with 224896 points. True


In [11]:
GoF = 5
key = 7 # index of the key frame to fit 
fitting_meshes = []
for i in range(5, 5 + GoF):
    if i==key:
        continue
    decimated_mesh_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(deformed_meshes[i], 10000, boundary_weight= 8000)
    print(decimated_mesh_i)
    fitting_mesh_i = subdivide_surface_fitting(decimated_mesh_i, deformed_meshes[key], 1)
    print(fitting_mesh_i)
    fitting_mesh_i.compute_vertex_normals()
    fitting_meshes.append(fitting_mesh_i)
fitting_meshes.append(deformed_meshes[key])
o3d.visualization.draw_geometries(fitting_meshes)

TriangleMesh with 5013 points and 10000 triangles.
TriangleMesh with 20034 points and 40000 triangles.
TriangleMesh with 20034 points and 40000 triangles.
TriangleMesh with 5013 points and 9999 triangles.
TriangleMesh with 20033 points and 39996 triangles.
TriangleMesh with 20033 points and 39996 triangles.
TriangleMesh with 5017 points and 9999 triangles.
TriangleMesh with 20041 points and 39996 triangles.
TriangleMesh with 20041 points and 39996 triangles.
TriangleMesh with 5019 points and 10000 triangles.
TriangleMesh with 20047 points and 40000 triangles.
TriangleMesh with 20047 points and 40000 triangles.


In [12]:
all_meshes = o3d.geometry.TriangleMesh()
for i in range(0,GoF):
    all_meshes += fitting_meshes[i]
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pre_reference_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(reference_pcd, depth=9, linear_fit=True)
print(pre_reference_mesh)
pre_reference_mesh.compute_vertex_normals()

pre_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
#o3d.visualization.draw_geometries([pre_reference_mesh])

reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(pre_reference_mesh, 40000, boundary_weight= 8000)
print(reference_mesh)
reference_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([reference_mesh])
o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch/reference_mesh/reference_mesh_{GoF:02}.obj', reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

TriangleMesh with 95148 points and 189992 triangles. True
PointCloud with 95148 points. True
run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 95148 / 14472
[Open3D DEBUG] #   Got kernel density: 0.013 (s), 351.938 (MB) / 416.637 (MB) / 606 (MB)
[Open3D DEBUG] #     Got normal field: 0.235 (s), 383.09 (MB) / 416.637 (MB) / 606 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 9.307476e-06 / 8.855877e-01
[Open3D DEBUG] #       Finalized tree: 0.152 (s), 427.41 (MB) / 427.41 (MB) / 606 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.096 (s), 408.867 (MB) / 427.41 (MB) / 606 (MB)
[Open3D DEBUG] #Set point constraints: 0.03 (s), 388.84 (MB) / 427.41 (MB) / 606 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 886754 / 1003360 / 10073
[Open3D DEBUG] Memory Usage: 388.840 MB
[Open3D DEBUG] # Linear system solved: 0.389 (s), 430.5 (MB) / 430.5 (MB) / 606 (MB)
[Open3D DEBUG] Got average: 0.00399995 (s), 411.305 (MB) / 430.5 (MB) / 606 (MB)
[Open3D DEBUG] Iso-V

True

## remember to adjust file path

In [13]:
load_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch/reference_mesh/reference_mesh_{GoF:02}.obj', enable_post_processing = False)
print(load_reference_mesh)

o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch/reference_mesh/reference_mesh_{GoF:02}.obj', load_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
load_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch/reference_mesh/reference_mesh_{GoF:02}.obj', enable_post_processing = False)
print(load_reference_mesh)
reference_vertices = np.array(load_reference_mesh.vertices)
load_reference_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([load_reference_mesh])
load_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
print(reference_vertices)

TriangleMesh with 20006 points and 40000 triangles.
TriangleMesh with 20006 points and 40000 triangles.
[[ 0.42036399  1.14646995 -0.46491101]
 [ 0.41304499  1.14443994 -0.46144599]
 [ 0.42036399  1.15957999 -0.46789399]
 ...
 [-0.141303    0.103927   -0.0238941 ]
 [ 0.27610001  1.62351    -0.00624901]
 [-0.247096    1.26523995  0.28488499]]


In [14]:
load_reference_mesh_trimesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch/reference_mesh/reference_mesh_{GoF:02}.obj', enable_post_processing = False)

print(np.array(load_reference_mesh_trimesh.vertices).__len__())
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(load_reference_mesh_trimesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)

subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/subdivided_reference_mesh.obj', subdivided_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


20006
TriangleMesh with 5006 points and 10000 triangles.
TriangleMesh with 20006 points and 40000 triangles.


True

## Run mesh editing now